In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel

### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [10]:
path_in = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in, path_out)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [9]:
wwl = WasWordtLijst.from_excel(path_out)
print(wwl.df)

Alle SBB codes zijn valide
Alle VvN codes zijn valide
         VvN    SBB
0     14rg10   14-l
1        NaN    7-a
2        NaN   8b-b
3        NaN   11/c
4        NaN  11a-b
...      ...    ...
1127     NaN   43-b
1128     NaN   43-d
1129     NaN   42-d
1130     NaN  43c1j
1131     NaN  41a3d

[1132 rows x 2 columns]


### Definitietabel opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [11]:
path_in = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_out = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in, path_out)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


In [12]:
dt = DefinitieTabel.from_excel(path_out)
print(dt.df)

Alle SBB codes zijn valide
Alle VvN codes zijn valide
     HABCODE Kwaliteit     VvN  \
0      H1130         G    2aa1   
1      H1130         G    2aa2   
2      H1130         G    3aa1   
3      H1130         G    3aa2   
4    H1140_A         G    2aa1   
..       ...       ...     ...   
628    H91F0         G  17aa1b   
629    H91F0         G   37ab1   
630    H91F0         G   43aa1   
631    H91F0         M   43rg2   
632    H91F0         M     NaN   

                                                  mits  \
0    mits in de FGR Getijdengebied en mits gelegen ...   
1    mits in de FGR Getijdengebied en mits gelegen ...   
2    mits in de FGR Getijdengebied en mits gelegen ...   
3    mits in de FGR Getijdengebied en mits gelegen ...   
4    mits in de slikkige en fijnzandige delen van F...   
..                                                 ...   
628                                                NaN   
629                                                NaN   
630            